In [1]:
!pip install transformers==4.44.0
!pip install accelerate==1.2.1
!pip install python-dotenv==1.0.1
!pip install requests==2.31.0
!pip install PyYAML==6.0
!pip install Jinja2==3.1.0
!pip install tornado==6.2
!pip install outlines==0.1.12
!pip install datasets
!pip install sentencepiece
!export HF_TOKEN="YOUR_HF_TOKEN"

  Using cached accelerate-1.2.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.2.1-py3-none-any.whl (336 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (2.0 kB)
Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2
  Using cached Jinja2-3.1.0-py3-none-any.wh

In [2]:
!pip install --upgrade transformers


  Using cached transformers-4.48.2-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.48.2-py3-none-any.whl (9.7 MB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0


In [3]:
import os
import requests          # For making API calls
import json              # For parsing and working with JSON data
import datetime          # For handling dates and validating input
from IPython import get_ipython
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import ipywidgets as widgets
from dotenv import load_dotenv
import outlines
import datasets
import torch


In [4]:
from dotenv import load_dotenv
import outlines

load_dotenv()

def entityIdForFlights(place):
    url = "https://sky-scanner3.p.rapidapi.com/flights/auto-complete"
    
    querystring = {"query":place}
    
    headers = {
    	"x-rapidapi-key": "b611647312msh00816fd3d25c377p15672djsn9277a7b612c3",
    	"x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    return response.json()['data'][0]['presentation']['skyId']

In [5]:
from dotenv import load_dotenv
import outlines

load_dotenv()

def entityIdForHotels(place):
        url = "https://sky-scanner3.p.rapidapi.com/hotels/auto-complete"
        
        querystring = {"query": place}
        
        headers = {
        	"x-rapidapi-key": "b611647312msh00816fd3d25c377p15672djsn9277a7b612c3",
        	"x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
        }
        
        response = requests.get(url, headers=headers, params=querystring)
        
        return response.json()['data'][0]['entityId']

In [6]:
from dotenv import load_dotenv
import outlines

load_dotenv()

def search_flights(Origin, destination, start_date, end_date):
    """
    Queries Flight Scaper's roundtrip flight search API.
    """
    url = "https://flights-scaper.p.rapidapi.com/flights/search-roundtrip"
    headers = {
      "x-rapidapi-key": "b611647312msh00816fd3d25c377p15672djsn9277a7b612c3",
      "x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    params = {"fromEntityId": Origin,"ToEntityId":destination,"departDate":start_date,"returnDate":end_date,"adults":"1"}

    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [7]:
from dotenv import load_dotenv
import outlines

load_dotenv()

def search_hotels(location, check_in, check_out):
    """
    Queries Flight Scaper's hotel search API.
    """
    url = "https://flights-scaper.p.rapidapi.com/hotels/search"
    headers = {
      "x-rapidapi-key": "b611647312msh00816fd3d25c377p15672djsn9277a7b612c3",
      "x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    params = {"entityId":location,"checkin":check_in,"checkout":check_out,"rooms":"1","adults":"1"}

    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [8]:
from transformers import AutoTokenizer, pipeline

# Initialize the model used as ChatBot
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Hermes-3-Llama-3.2-3B")
llama_model = pipeline("text-generation", model="NousResearch/Hermes-3-Llama-3.2-3B", device=0)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [9]:
schema = {
        "type": "object",
        "properties": {
            "Origin": {"type": "string", "default": None},
            "destination": {"type": "string", "default": None},
    "Travel_start_date": {
      "oneOf": [
        {"type": "string", "format": "date"},
        {"enum": ["N/A"]},
        {"type": "null"}
      ],
      "default": None
    },
    "Travel_end_date": {
      "oneOf": [
        {"type": "string", "format": "date"},
        {"enum": ["N/A"]},
        {"type": "null"}
      ],
      "default": None
    },
            "Budget": {"type": "string", "default": None},
            "Hotel_Preferences": {"type": "string", "default": None},
            "Flight_Preferences": {"type": "string", "default": None}
        },
            "required": ["Origin", "Destination", "Travel_start_date", "Travel_end_date"]
            
    }
model = outlines.models.transformers("ibm-granite/granite-3.1-3b-a800m-instruct")
        
# Convert schema to JSON string
schema_json = json.dumps(schema)

generator = outlines.generate.json(model, schema_json) 


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
import json


class LlamaTravelBot:
    
    def __init__(self,tokenizer,model,generator):
        self.travel_details = {}  # Store travel details
        self.tokenizer = tokenizer
        self.model = model
        self.generator=generator
        self.chat_history = [
    {
        "role": "system",
        "content": (
            "You are an intelligent and conversational travel assistant. "
            "The current year is 2025. Your goal is to help the user plan their trip. "
            "1. Have a natural conversation to get the following details from the user: "
            "Required: Origin, Destination, Travel Start Date, Travel End Date. "
            "Optional: Budget, Hotel Preferences (e.g., location, amenities), "
            "Flight preferences (e.g., airline, class). "
            "2. Dynamically adjust the conversation based on missing details. "
            "3. Do not assume information; only take into account what the user provides."
        ),
    },
    {
        "role": "assistant",
        "content": "Hi! I'm your travel assistant. How can I help you today?",
    },
]

        self.details={}

    def update_travel_details(self):
         """
        Update the travel details dictionary with new information.
         """
         self.travel_details.update(self.details)
       

    def llama_ask(self, prompt):
        """
        Generate a response from the Llama model.
        """
        self.chat_history.append({"role":"assistant","content":" "})
                
        input_text ="\n".join([f"{msg['role']}: {msg['content']}" for msg in self.chat_history])
        
        # Call the model with the adjusted input
        output = self.model(input_text, max_new_tokens=512, do_sample=True, top_k=30, top_p=0.9, temperature=0.7,stop_strings=["<|eot_id|>","\nuser","\nsystem"],tokenizer=self.tokenizer)
          # Cumulative probability for nucleus sampling
        relevant_output=output[0]['generated_text']
        last_assistant_index = relevant_output.rfind("\nassistant:")  # Find the last occurrence of '\nassistant:'
        if last_assistant_index != -1:
            response_start = last_assistant_index + len("\nassistant:")
            response_end = relevant_output.find("\nuser", response_start)  # Find the next '\nuser' after '\nassistant:'
            response = relevant_output[response_start:response_end].strip() if response_end != -1 else relevant_output[response_start:].strip()
        
        for msg in self.chat_history:
            if msg["role"] == "assistant" and msg["content"].strip() == "":
                msg["content"] = response
                break

        # Return only the newest response
        return response

    def handle_user_input(self):
        """
        Process user input and manage the conversation flow.
        """
        print("Bot: Hi! I'm your travel assistant. How can I help you today?")
    
        while True:
            while not self.is_ready_for_api():              
                user_input = input("You: ").strip()  
    
                if user_input.lower() in ["exit", "quit"]:                   
                    print("Bot: Goodbye! Safe travels!")
                    return  # Use return instead of break to exit the method
                
                # Add user input to the chat history
                self.chat_history.append({"role":"user","content":user_input})
                
                
                llama_response = self.llama_ask(user_input)
                print(f"Bot: {llama_response}") 
                # Append the new response to the chat history
                self.chat_history.append({"role": "assistant", "content": llama_response})             
                self.details = self.extract_details_from_input(user_input)
                
                self.update_travel_details()
                print(self.travel_details)
    
            if self.is_ready_for_api():
                while True:
                    print("Bot: I have gathered all the necessary details. Would you like to proceed with your request and show you the available flights and hotels? (Y/N)")
                    self.chat_history.append({"role": "assistant", "content": "I have gathered all the necessary details. Would you like to proceed with your request and show you the available flights and hotels? (Y/N)"})
                    user_input = input("User: ")
                    
                    if user_input.upper() == "Y":
                        print("Bot: Great! Fetching the available flights and hotels for you...")
                        self.fetch_and_present_results()
                        return  # Exit the method after fetching and presenting results
                    
                    elif user_input.upper() == "N":
                        print("Bot: No problem! Please let me know what additional details you'd like to provide.")
                        self.chat_history.append({"role": "assistant", "content": "No problem! Please let me know what additional details you'd like to provide."})
                        
                        additional_input = input("You: ").strip()  
                        self.chat_history.append({"role": "user", "content": additional_input})
                        
                        # Extract additional details and update the travel details
                        self.details = self.extract_details_from_input(additional_input)
                        self.update_travel_details()
                        print(self.travel_details)
                    
                    else:
                        print("Bot: Oops! It seems there was a misunderstanding. Please enter 'Y' to proceed or 'N' to provide more details.")
                        self.chat_history.append({"role": "assistant", "content": "Oops! It seems there was a misunderstanding. Please enter 'Y' to proceed or 'N' to provide more details."})
    

    def extract_details_from_input(self, user_input):
        """
        Use the LLM to analyze user input and extract key details.
        """
        
        prompt = f"""I want you to analyze and extract the following details from the chat history between a Travel agent and a customer
        Here's the chat history so far: '{self.chat_history}'.


        Details to be extracted:
        Required: Origin (the origin is where the user or customer is presently located or resides), Destination (the travel/trip location), Travel Start Date, Travel End Date (for context The current year is 2025, so if the year isn't mentioned in the conversation assume it to be 2025. Just make sure you dont assume the dates just cause you know the year.).
        Optional: Budget, Hotel Preferences, (e.g., location, amenities), Flight preferences (e.g., airline, class)
        
        Provide a JSON object according to the following schema:
        - Origin: (type:string) 
        - destination: (type:string)
        - Travel_start_date: (type:string) (in YYYY-MM-DD format)
        - Travel_end_date: (type:string) (in YYYY-MM-DD format)
        - budget: (type:string)
        - Hotel Preferences: (type:string)
        - Flight Preferences: (type:string)

        If any of the above listed details are not provided you **MUST** set it to None .
        Do not assume information; only take into account what the user provides
        """

         # Use the JSON string
        response = self.generator(prompt)

        try:
            if isinstance(response, dict):
                response = json.dumps(response)

            # Parse the JSON response from the LLM
            extracted_details = json.loads(response)
            return extracted_details
        except json.JSONDecodeError:
            print("Error parsing the LLM's response.")
            return {}


    def is_ready_for_api(self):
        """
        Check if all required fields are collected and contain valid values.
        """
        required_fields = ["Origin", "destination", "Travel_start_date", "Travel_end_date"]
        invalid_values = {"N/A", "null", "none", "Not Provided",""}  # Set of invalid values (case-insensitive)

        # Ensure all required fields are present and contain valid values
        return all(
            field in self.travel_details
            and str(self.travel_details[field]).strip().lower() not in invalid_values
            for field in required_fields
        )
    def fetch_and_present_results(self):
        """
        Use APIs to fetch results and present them to the user.
        """
        Origin = self.travel_details["Origin"]
        destination = self.travel_details["destination"]
        start_date = self.travel_details["Travel_start_date"]
        end_date = self.travel_details["Travel_end_date"]
        budget = self.travel_details.get("budget", "No budget specified")

        print(Origin)
        print(destination)
        print(start_date)
        print(end_date)
        print(budget)
        
        originid= entityIdForFlights(Origin)
        destinationid1= entityIdForFlights(destination)
        destinationid2= entityIdForHotels(destination)

        print(originid)
        print(destinationid1)
        print(destinationid2)
        print(type(originid))
        print(type(destinationid1))
        print(type(destinationid2))
        
        # Fetch flight and hotel data
        flights = search_flights(originid, destinationid1, start_date, end_date)
        print(flights)

        # Present results
        print("\nFlights:")
           
        top_itineraries = flights['data']['itineraries']
        
        # Iterate over the top 3 itineraries
        for i in range(min(3, len(top_itineraries))):
            itinerary = top_itineraries[i]
            price = itinerary['price']['formatted']
            
            print(f"*** Itinerary {i + 1} ***")
            print(f"Price: {price}")
            
            # Loop through each leg in the itinerary
            for j, leg in enumerate(itinerary['legs']):
                origin = leg['origin']['name']
                destination = leg['destination']['name']
                duration = leg['durationInMinutes']
                departure = leg['departure']
                arrival = leg['arrival']
                
                # Determine whether it's outbound or return
                flight_type = "Outbound flight" if j == 0 else "Return flight"
                
                print(f"{flight_type}:")
                print(f"  Origin airport: {origin}")
                print(f"  Destination airport: {destination}")
                print(f"  Duration: {duration} minutes")
                print(f"  Departure time: {departure}")
                print(f"  Arrival time: {arrival}")
            print("\n")
                                                                                    
      # Sample code to iterate through top 3 hotels and showcase desired details
        hotels = search_hotels(destinationid2, start_date, end_date)
        print(hotels)
        hotel_cards = hotels['data']['results']['hotelCards']
        print('\nHotels:')        
        # Iterate over the top 3 hotels
        for i, hotel in enumerate(hotel_cards[:3]):
            hotel_name = hotel['name']
            review_score = hotel['reviewsSummary']['score']
            location = hotel['relevantPoiDistance']
            cheapest_price = hotel['cheapestPrice']
        
            print(f"Hotel {i + 1}:")
            print(f"  Name: {hotel_name}")
            print(f"  Review Score: {review_score}")
            print(f"  Location : {location}")
            print(f"  Cheapest Price Available: {cheapest_price}")
            print("-" * 30)

# Interactive session
if __name__ == "__main__":
    bot = LlamaTravelBot(tokenizer, llama_model,generator)
    bot.handle_user_input()



Bot: Hi! I'm your travel assistant. How can I help you today?


You:  Hii can you help me plan a trip to paris


Bot: Sure, I'd be happy to help plan your trip to Paris! When would you like to go?
{'Origin': '', 'destination': 'Paris', 'Travel_start_date': '2025-01-01', 'Travel_end_date': '2025-12-31', 'Budget': 'N/A', 'Hotel_Preferences': 'N/A', 'Flight_Preferences': 'N/A'}


You:  Ideally sometime in the summer, so from 10th june to the 15th


Bot: Great, you want to go to Paris in the summer, from June 10th to June 15th. Is there a specific budget you're working with?
{'Origin': '', 'destination': 'Paris', 'Travel_start_date': '2025-06-10', 'Travel_end_date': '2025-06-15', 'Budget': 'N/A', 'Hotel_Preferences': 'N/A', 'Flight_Preferences': 'N/A'}


...

Bot: No problem! Please let me know what additional details you'd like to provide.


You:  My origin is new york and my destination is paris


{'Origin': 'New York', 'destination': 'Paris', 'Travel_start_date': '2025-06-10', 'Travel_end_date': '2025-06-15', 'Budget': '$1200', 'Hotel_Preferences': 'None', 'Flight_Preferences': 'Quickest flights with no layovers'}
Bot: I have gathered all the necessary details. Would you like to proceed with your request and show you the available flights and hotels? (Y/N)


User:  Y


Bot: Great! Fetching the available flights and hotels for you...
New York
Paris
2025-06-10
2025-06-15
No budget specified
NYCA
PARI
27539733
<class 'str'>
<class 'str'>
<class 'str'>
{'data': {'context': {'status': 'incomplete', 'sessionId': 'Cl0IARJZCk4KJGU4MmMwYTAwLWNjMDMtNGRiMi1hNTY0LThmMzhmNDM1MGYwZBACGiQxNmFkN2IzNi0xYjJmLTRiY2MtODQyZC1lODQ2NGIzODdlNGIQnPvRys0yGAESKHVzc19hYjk3MTk4Mi00YjU0LTRkZjktOGZjYS1kM2FhZTAzYTExYTkiAlVT', 'totalResults': 10}, 'itineraries': [{'id': '11442-2506102300--30858-0-15083-2506111215|15083-2506151850--30858-0-11442-2506152100', 'price': {'raw': 625.91, 'formatted': '$626', 'pricingOptionId': 'n23AAsxxKDtf'}, 'legs': [{'id': '11442-2506102300--30858-0-15083-2506111215', 'origin': {'id': 'EWR', 'entityId': '95565059', 'name': 'New York Newark', 'displayCode': 'EWR', 'city': 'New York', 'country': 'United States', 'isHighlighted': False}, 'destination': {'id': 'ORY', 'entityId': '95565040', 'name': 'Paris Orly', 'displayCode': 'ORY', 'city': 'Paris', 'coun